In [1]:
#Opomba: Mnozica I predstavlja najvecjo mnozico neodvisnih vozlisc
import time
import random
import csv

# CLP =================================================================================================================================
def clp(graf):
    zac = time.time() #Zacetek merjenja casa

    CLP = MixedIntegerLinearProgram(maximization = True) #CLP, ki ga bomo maximizirali
    vozlisca = CLP.new_variable(binary = True) #Vsakemu vozliscu priredimo vrednost 1 ali 0 (binarne spremenljivke)
    CLP.set_objective(sum([vozlisca[v] for v in graf.vertices()])) #Definiramo kriterijsko funkcijo

    for u,v in graf.edges(labels = False):
        CLP.add_constraint(vozlisca[u] + vozlisca[v] <= 1) #Prvi pogoj CLP

    moc_max = CLP.solve() #resi CLP in nam pove Moc mnozice I
    vozlisca = CLP.get_values(vozlisca) #za vsako vozlisce dobimo njegovo vrednost x_v

    #Seznam vozlisc v mnozici I:
    sez = [k for k, v in vozlisca.items() if v == 1.0]

    #Merjenje casa:
    kon = time.time()
    cas = kon - zac
    return [int(moc_max), sez, cas]


# Relaksacija CLP ==========================================================================================================================
def rclp(graf):
    zac = time.time() #Zacetek merjenja casa

    RCLP = MixedIntegerLinearProgram(maximization = True) #Relak. CLP, ki ga bomo maximizirali
    vozlisca = RCLP.new_variable(real = True) #Vsakemu vozliscu priredimo vrednost 1 ali 0
    RCLP.set_max(vozlisca,1)
    RCLP.set_min(vozlisca,0)
    RCLP.set_objective(sum([vozlisca[v] for v in graf.vertices()])) #Kriterijska funkcija

    for u,v in graf.edges(labels = False):
        RCLP.add_constraint(vozlisca[u] + vozlisca[v] <= 1) #Prvi pogoj

    RCLP.solve()
    vozlisca = RCLP.get_values(vozlisca)

    #Seznam vozlisc v mnozici I:
    sez = [k for k, v in vozlisca.items() if v > 0.5]

    #Moc mnozice I:
    moc_max = len(sez)

    #Merjenje casa:
    kon = time.time()
    cas = kon - zac
    return [int(moc_max), sez, cas]


# Lokalno iskanje ==========================================================================================================================
def lokalno_iskanje(graf, st_ponovitev):
    moc_max = 0 #moc največje neodvisne mnozice vozlisc
    max_mnozica_neod = [] #največja mnozica neodvisnih vozlisc
    zac = time.time()

    for i in range(st_ponovitev):
        mnozica_neod = set() #mnozica neodvisnih vozlisc
        mnozica_neod.add(graf.random_vertex()) #dodamo nakljucno vozlisce v seznam_neod
        for v in random.sample(graf.vertices(), len(graf.vertices())):
            if not any(u in mnozica_neod for u in graf[v]): # ali je kateri od sosedov vozlišča v v množici? #VPRASANJE: Kaj pomeni graf[v]
                    mnozica_neod.add(v)
        if len(mnozica_neod) > moc_max: #iscemo najvecjo mnozico neodvisnih vozlisc
            max_mnozica_neod = mnozica_neod
            moc_max = len(mnozica_neod)

    #Merjenje casa:
    kon = time.time()
    cas = kon - zac
    return[moc_max, max_mnozica_neod, cas]

In [2]:
# Funkcija za testiranje vseh treh algoritmov na nakljucnem grafu:
# testiraj(stevilo vozlisc, verjetnost povezave med dvema vozliscema, stevilo ponovitev lokalnega iskanja) =======================================
def testiraj(st_vozlisc, verjetnost, st_ponovitev=10):
    rezultati = []
    graf = graphs.RandomGNP(st_vozlisc, verjetnost)
    rezultati.append(clp(graf))
    rezultati.append(rclp(graf))
    rezultati.append(lokalno_iskanje(graf, st_ponovitev))
    return sum(rezultati, []) #Seznam seznamov se stakne v en sam seznam

In [3]:
testiraj(6,0.6,1)

[3,
 [1, 2, 4],
 0.04898214340209961,
 1,
 [4],
 0.0009739398956298828,
 3,
 {1, 2, 4},
 0.0007917881011962891]

In [4]:
# Funkcija za izpis v CSV v mapo 'Rezultati_testov' (izvede zgornjo funkcijo 'testiraj' na grafih s številom vozlisc od 1 do max_stevilo_vozlisc, pri dolocenih verjetnostih povezave dveh vozlisc in dolocenem stevilu ponovitev lokalnega iskanja):
def izpis_csv(max_stevilo_vozlisc, verjetnost, st_ponovitev):
    trenutni_cas = time.strftime("%Y-%m-%d_%H-%M-%S")
    with open('Rezultati_testov/test_{}.csv'.format(trenutni_cas), 'wb') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=';',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['ST_VOZLISC', 'VERJETNOST', 'ST_PONOVITEV_LOKISK',
                             'CLP_MAX_MOC', 'CLP_VOZLISCA_I', 'CLP_CAS', 
                             'RCLP_MAX_MOC', 'RCLP_VOZLISCA_I', 'RCLP_CAS',
                             'LOKISK_MAX_MOC', 'LOKISK_VOZLISCA_I', 'LOKISK_CAS']) #zacena vrstica z imeni stolpcev
        for i in range(1, max_stevilo_vozlisc + 1):
            filewriter.writerow(sum([[i], [verjetnost], [st_ponovitev], testiraj(i, verjetnost, st_ponovitev)], []))

In [5]:
izpis_csv(50, 0.3, 15)